# users table에 패스워드 넣기

In [1]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [2]:
import json
with open(filename) as fp:
  config_str = fp.read()
config = json.loads(config_str)

In [3]:
import pymysql 
conn = pymysql.connect(
 host = config['host'],
 user = config['user'],
 password = config['password'],
 database = config['database'],
 port = config['port']
 )

## 패스워드를 암호화

* Secure Hash Algorithm(SHA=256)

In [5]:
import hashlib

pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

* base64로 인코딩

In [9]:
import base64

base64.b64encode(pwd_sha256.digest())   # 결과가 byte형식

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [10]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd                                                            # 결과가 string

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [11]:
len(hashed_pwd)

44

## 패스워드 생성하는 함수 만들기

In [12]:
import hashlib, base64

def gen_pwd(pwd):
 pwd_sha256 = hashlib.sha256(pwd.encode())
 hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
 return hashed_pwd

In [13]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## users table에 패스워드 넣기

* uid + 21

In [15]:
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('gdhong2',),
 ('jbpark',),
 ('jbpark2',),
 ('wjlee',))

In [21]:
cur.execute(sql)
for result in cur:
  pwd = f'{result[0]}21'
  print(gen_pwd(pwd))

admin21
djy21
eskim21
gdhong21
gdhong221
jbpark21
jbpark221
wjlee21


In [23]:
sql_update = 'update users set pwd=%s where uid=%s;'
for line in results:
  uid = line[0]
  pwd = gen_pwd(uid+'21')
  cur.execute(sql_update, (pwd, uid))
conn.commit()

* email data update : uid@mc.com

In [24]:
sql_email = 'update users set email=%s where uid=%s;'
for line in results:
  uid = line[0]
  email = uid +'@mc.com'
  cur.execute(sql_email, (email, uid))
conn.commit()

## 로그인

1) uid가 없는 경우

In [55]:
sql_login = 'select uid, pwd from users where uid=%s and is_deleted=0'

In [56]:
# result가 null이면 uid가 없는경우

uid = 'fool'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

2) 비밀번호가 틀린 경우

In [57]:
uid, pwd = 'admin', 'admin23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result

('admin', 'vZoHtIRE7hlNqRQhwDu2J7/8EtyuBCAblH9heH/JKOo=')

In [58]:
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
  print('Login')
else:
  print('Incorrect passward')

Incorrect passward


3) uid, pwd가 맞은경우

In [61]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd = result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else:
    print('Incorrect password!')

Login


In [66]:
uid, pwd = 'djy1', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
else:
    print('Bad uid!!!')

Bad uid!!!


* 예외 처리를 통해 해결하는 경우

In [67]:
uid, pwd = 'djy', 'djy21'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else:
        print('Incorrect password!')
except:
    print('Bad uid!!!')

Login


In [68]:
cur.close()
conn.close()